#### Get canada data from Wikipedia and start exploring.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(response.text, 'lxml')

table = soup.find('table', {'class':'wikitable sortable'})
table_rows = table.find_all('tr')

canada_list = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    canada_list.append(row)
    
df = pd.DataFrame(canada_list, columns=['PostalCode', 'Borough', 'Neighbourhood'])

df['Neighbourhood'] = df['Neighbourhood'].str.strip()

df.Borough.replace('Not assigned', np.nan, inplace=True)

df.Neighbourhood.replace('Not assigned', df['Borough'], inplace=True)

df.dropna(subset=['Borough'], inplace=True)

canada_df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()

canada_df.shape


(103, 3)

#### Load longitude and latitude from CSV file

In [2]:
geocode_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [3]:
clean_df = pd.merge(canada_df, geocode_df, left_on='PostalCode', right_on='Postal Code', how='left')

clean_df.drop(['Postal Code'], axis=1, inplace=True)

clean_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


#### Getting latitude and longitude of Toronto

In [4]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")

location = geolocator.geocode(address)

toronto_latitude = location.latitude
toronto_longitude = location.longitude

#### Create a map of Toronto

In [5]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

toronto_data = clean_df[clean_df['Borough'].str.contains('Toronto')]

map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

for index, row in toronto_data.iterrows():
    label = '{}, {}'.format(row.Neighbourhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.Latitude, row.Longitude],
        radius=5,
        label=label,
        color='green',
        fill=True,        
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

In [12]:
CLIENT_ID = '0SFWND5UEBSAUFKQHV15VINMGVUT5CGTP42VZIJC52GCOKQG'
CLIENT_SECRET = 'BV22D1J3PF05NUM0DWJIZCBKP1PYEFGWG0513R1DKR5J2ONX'
VERSION = '20190625'
LIMIT = 100

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
#toronto_venues = getNearbyVenues(names=toronto_data['PostalCode'],
#                                   latitudes=toronto_data['Latitude'],
#                                  longitudes=toronto_data['Longitude'])
toronto_venues.shape
toronto_venues.to_csv(index=True)

',Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category\n0,M4E,43.67635739999999,-79.2930312,Glen Manor Ravine,43.67682094413784,-79.29394208780985,Trail\n1,M4E,43.67635739999999,-79.2930312,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store\n2,M4E,43.67635739999999,-79.2930312,Grover Pub and Grub,43.679181434941015,-79.29721535878515,Pub\n3,M4E,43.67635739999999,-79.2930312,Glen Stewart Ravine,43.67629984029563,-79.2947841389563,Other Great Outdoors\n4,M4E,43.67635739999999,-79.2930312,Upper Beaches,43.68056321147582,-79.2928688743688,Neighborhood\n5,M4K,43.6795571,-79.352188,Pantheon,43.67762124481265,-79.35143390043564,Greek Restaurant\n6,M4K,43.6795571,-79.352188,Dolce Gelato,43.677772998450614,-79.35118737317053,Ice Cream Shop\n7,M4K,43.6795571,-79.352188,MenEssentials,43.677820068604575,-79.35126543045044,Cosmetics Shop\n8,M4K,43.6795571,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant\n9

In [72]:
toronto_venues.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [79]:
toronto_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

print (toronto_hot.shape)

fixed_columns = [toronto_hot.columns[-1]] + list(toronto_hot.columns[:-1])

toronto_hot = toronto_hot[fixed_columns]

toronto_hot['Postcode'] = toronto_venues['Postcode']

toronto_hot.shape

(1700, 238)


(1700, 239)

In [80]:
toronto_grouped = toronto_hot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,M4E,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,M4K,0.023810,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,M4L,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,M4M,0.026316,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,M4N,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,M4P,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,M4R,0.062500,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,M4S,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.026316,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,M4T,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,M4V,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.00


In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Pub,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,M4L,Park,Pizza Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
3,M4M,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant
4,M4N,Park,Swim School,Bus Line,Women's Store,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [86]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

postcode_venues_sorted['Cluster label'] = kmeans.labels_
#toronto_merged = pd.merge(postcode_venues_sorted, neighbourhoods, how='left', on='Postcode', validate="1:1")
postcode_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster label
0,M4E,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Pub,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,0
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Café,0
2,M4L,Park,Pizza Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,0
3,M4M,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant,0
4,M4N,Park,Swim School,Bus Line,Women's Store,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,3


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neigh, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Postcode'], toronto_merged['Cluster label']):
    label = folium.Popup(str(neigh) + '(' + str(pc) + '): Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters